In [ ]:
#ImportDependencies
import pandas as pd
import gmaps
import requests
%matplotlib inline
import numpy as np
from datetime import datetime

#Import the API key
from config import g_key

In [ ]:
#Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

In [ ]:
#Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
#Prompt the customer for the maximum temperature preference.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
#Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
city_rain = str(input("Do you want it to be raining? (yes/no) "))
#Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").
city_snow = str(input("Do you want it to be snowing? (yes/no) "))

In [ ]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp),
                                       (city_data_df["Max Temp"] >= min_temp),
                                       (city_data_df["Rain Inches (last 3 hrs)"] == yes),
                                       (city_data_df["Snow Inches (last 3 hrs)"] == yes)]
preferred_cities_df.head(10)

In [ ]:
#determine if there are any null values 
preferred_cities_df.count()

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp,coordinates, weather description.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df(10)

In [ ]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
current_description = hotel_df["Current Description"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4, current_description)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

In [ ]:
#Save and upload the new DataFrame as WeatherPy_vacation.csv.
output_data_file = "weather_data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
#Save and upload the new marker layer map as WeatherPy_vacation_map.png.
savefig(WeatherPy_vacation_map.png)